# Curso de Optimización (DEMAT)
## Tarea 4
##Leslie Janeth Quincosa Ramírez

| Descripción:                         | Fechas               |
|--------------------------------------|----------------------|
| Fecha de publicación del documento:  | **Febrero 24, 2022** |
| Fecha límite de entrega de la tarea: | **Marzo 6, 2022**    |


### Indicaciones

Puede escribir el código de los algoritmos que se piden en una
celda de este notebook o si lo prefiere, escribir las funciones
en un archivo `.py` independiente e importar la funciones para
usarlas en este notebook. Lo importante es que en el notebook
aparezcan los resultados de la pruebas realizadas y que:

- Si se requieren otros archivos para poder reproducir los resultados,
  para mandar la tarea cree un archivo ZIP en el que incluya
  el notebook y los archivos adicionales. 
- Si todos los códigos para que se requieren para reproducir los
  resultados están en el notebook, no hace falta comprimirlo 
  y puede anexar sólo el notebook en la tarea del Classroom.
- Exportar el notebook a un archivo PDF y anexarlo en la tarea del
  Classroom como un archivo independiente.
  **No lo incluya dentro del ZIP**, porque la idea que lo pueda accesar 
  directamente para poner anotaciones y la calificación de cada ejercicio.

En la descripción de los ejercicios se nombran algunas variables
para el algoritmo, pero sólo es para facilitar la descripción.
En la implementación pueden nombrar sus variables como gusten.

En los algoritmos se describen las entradas de las
funciones. La intención es que tomen en cuenta lo que requiere
el algoritmo y que tiene que haber parámetros que permitan
controlar el comportamiento del algoritmo,
evitando que dejen fijo un valor y que no se puede modificar
para hacer diferentes pruebas. Si quieren dar esta información
usando un tipo de dato que contenga todos los valores o
usar variables por separado, etc., lo pueden hacer y no usen
variables globales si no es necesario.

Lo mismo para los valores que devuelve una función. 
Pueden codificar como gusten la manera en que regresa los cálculos.
El punto es que podamos tener acceso a los resultados,
sin usar variables globales, y que la función no sólo imprima 
los valores que después no los podamos usar.


---

## Ejercicio 1 (5 puntos)

Programar el método de descenso máximo con tamaño de paso exacto para
minimizar funciones cuadráticas:

$$ f(x) = \frac{1}{2} x^\top \mathbf{A} x - b^\top x, $$

donde  $\mathbf{A} \in  \mathbb{R}^{n \times n}$ y $x \in \mathbb{R}^n$.

Dado el vector $b$,  la matriz $\mathbf{A}$, 
un punto inicial $x_0$, un número  máximo de iteraciones $N$, 
la tolerancia $\tau>0$. Fijar $k=0$ y repetir los siguientes pasos:

1. Calcular el gradiente en el punto $x_k$, 

   $$g_k = \nabla f(x_k) = \mathbf{A} x_k - b $$.
2. Si $\|g_k\| < \tau$, entonces $x_k$ es (casi) un punto estacionario.
   Hacer $res=1$ y terminar el ciclo. 
3. Elegir la dirección de descenso como $p_k = - g_k$.
4. Calcular el tamaño de paso $\alpha_{k}$ que minimiza el valor de la función
   $$\phi_k(\alpha) =  f(x_k + \alpha p_k)$$
   es decir, calcular
   $$ \alpha_{k} = -\frac{ g_{k}^{\top} p_{k}}{ p_{k}^{\top}\mathbf{A}p_{k}} $$
5. Calcular el siguiente punto de la secuencia como
   $$x_{k+1} = x_k + \alpha_k p_k $$
6. Si $k+1\geq N$, hacer $res=0$ y terminar.
7. Si no, hacer $k = k+1$ y volver el paso 1.
6. Devolver el punto $x_k$, $f_k= \frac{1}{2} x_k^\top \mathbf{A} x_k - b^\top x_k$, $g_k$, 
   $k$ y $res$.

---

1. Escriba una función que implementa el algoritmo anterior 
   usando arreglos de Numpy.  
2. Escriba una función para probar el funcionamiento del método
   de descenso máximo. Esta función debe recibir como parámetros
   el nombre de un archivo `.npy` que contiene las entradas de
   una matriz cuadrada $\mathbf{A}$, el vector $b$, un punto inicial $x_0$,
   el número máximo de iteraciones $N$ y la
   tolerancia $\tau$.   
   
* Esta función debe crear la matriz $\mathbf{A}$ y el vector $b$ 
  leyendo los archivos de datos.
* Obtener el número de filas $r$ de la matriz e imprimir este valor.
* Compruebe que la matriz es simétrica  y definida positiva calculando
  e imprimiendo el valor $\|\mathbf{A} - \mathbf{A}^\top\|$ y su
  eigenvalor más pequeño (use la función `numpy.linalg.eig()`).
* Ejecutar la función del Inciso 1.
* Dependiendo del valor de la variable $res$, imprima un mensaje que diga que el 
  algoritmo convergió ($res=1$) o no ($res=0$).
* Imprimir $k$, $f_k$, la norma de $g_k$, y los primeros 3 y últimos 3 elementos del arreglo $x_k$.
* Calcule directamente el minimizador resolviendo la ecuación 
  $Ax_* = b$ e imprima el valor del error $\|x_k - x_* \|$.
  
3. Pruebe la función del Inciso 2  usando $N=1000$, la tolerancia
   $\tau = \epsilon_m^{1/3}$, donde $\epsilon_m$ es el
   épsilon de la máquina, y los arreglos que se incluyen en 
   el archivo datosTarea04.zip, de la siguiente manera:

| Matriz   | Vector     | Punto para iniciar la secuencia  |
|----------|------------|----------------------------------|
| A1.npy   | b1.npy     |  $x_0 = (0, -5)$                 |
| A1.npy   | b1.npy     |  $x_0 = (7045, 7095)$            |
| A2.npy   | b2.npy     |  $x_0 = (0,0,...,0) \in \mathbb{R}^{500}$             |
| A2.npy   | b2.npy     |  $x_0 = (10000,10000,...,10000) \in \mathbb{R}^{500}$ |

   
### Solución:

In [37]:
# En esta celda puede poner el código de las funciones
# o poner la instrucción para importarlas de un archivo .py

import numpy as np
from numpy import linalg as LA
from numpy.linalg import eigvals
import sys

def DescMAxPaso (b, A, N, x0, tol):
    xk = x0      
    for k in range(N):
      gk = A@xk - b
      norm = LA.norm(gk)
      if norm < tol:
          res = 1
          break
      else:
          pk = -gk
          ak = - (gk.T@pk)/(pk.T@A@pk)
          xk = xk + ak*pk
          if k+1 >= N:
              res = 0
              break
    fk = (1/2)*xk.T@A@xk-b.T@xk
    return xk, fk, gk, k, res
    


In [38]:
# Lectura de datos y pruebas realizadas

def Probar(archivo_A, archivo_b, x0, N, tol):
    A = np.load(archivo_A)
    b = np.load(archivo_b)
    r = A.shape[0]
    print('Número de filas:',r)
    eigval = np.linalg.eigvals(A)
    eigmin = eigval.min()
    a = LA.norm(A - A.T)
    print('Eigenvalores:', eigval)
    print('||A-AT||:', a)
    xk, fk, gk, k, res = DescMAxPaso (b, A, N, x0, tol)
    if res == 1:
        print('¡Convergió! :)')
    else:
        print('¡No convergió! :(')
    print('Valor k:', k)
    print('fk:', fk)
    print('||gk||:', LA.norm(gk))
    print('Vector xk:', xk[:3], '...', xk[-3:])
    x = np.linalg.solve(A, b)
    print('||xk-x||:', LA.norm(xk-x))




In [39]:
#Pruebas

Probar('A1.npy', 'b1.npy', np.array([0,-5]), 1000, tol)

Número de filas: 2
Eigenvalores: [1.  0.1]
||A-AT||: 0.0
¡Convergió! :)
Valor k: 69
fk: -62.749999999933024
||gk||: 4.767340294300877e-06
Vector xk: [-24.49997287  25.49997743] ... [-24.49997287  25.49997743]
||xk-x||: 3.528998540703229e-05


In [40]:
# Prueba: Descenso máximo con paso exacto

from numpy.core.fromnumeric import partition
#Pruebas
eps = sys.float_info.epsilon
print('epsilon de la maquina', eps)
tol = eps**(1/3)

x0s = [np.array([0, -5]) , np.array([7045,7095])]


print("Descenso máximo con paso exacto. \n")

for x0 in x0s:
      Probar('A1.npy', 'b1.npy', x0, 1000, tol)
      print('\n')


epsilon de la maquina 2.220446049250313e-16
Descenso máximo con paso exacto. 

Número de filas: 2
Eigenvalores: [1.  0.1]
||A-AT||: 0.0
¡Convergió! :)
Valor k: 69
fk: -62.749999999933024
||gk||: 4.767340294300877e-06
Vector xk: [-24.49997287  25.49997743] ... [-24.49997287  25.49997743]
||xk-x||: 3.528998540703229e-05


Número de filas: 2
Eigenvalores: [1.  0.1]
||A-AT||: 0.0
¡Convergió! :)
Valor k: 1
fk: -62.75
||gk||: 6.425429159208664e-13
Vector xk: [-24.5  25.5] ... [-24.5  25.5]
||xk-x||: 9.131096203816022e-13




Nota: el vector $x_k$ solo tiene dos entradas.

In [42]:
#Prueba: A2.npy	b2.npy	 x0=(0,0,...,0)∈R500

print("Descenso máximo con paso exacto. \n")
x0s = [np.zeros(500) , 1000*np.ones(500)]

for x0 in x0s:
      Probar('A2.npy', 'b2.npy', x0, 1000, tol)
      print('\n')



Descenso máximo con paso exacto. 

Número de filas: 500
Eigenvalores: [5.         4.99018036 4.98036072 4.97054108 4.96072144 4.9509018
 4.94108216 4.93126253 4.92144289 4.91162325 4.90180361 4.89198397
 4.88216433 4.87234469 4.86252505 4.85270541 4.84288577 4.83306613
 4.82324649 4.81342685 4.80360721 4.79378758 4.78396794 4.76432866
 4.7741483  0.1        0.10981964 0.11963928 0.12945892 0.13927856
 0.1490982  0.15891784 0.16873747 0.17855711 0.18837675 0.19819639
 0.20801603 0.21783567 0.22765531 0.23747495 0.24729459 0.25711423
 0.26693387 0.27675351 0.28657315 0.33567134 0.29639279 0.3258517
 0.31603206 0.30621242 4.75450902 4.74468938 4.73486974 4.7250501
 4.68577154 4.71523046 4.70541082 4.69559118 0.34549098 0.35531062
 0.40440882 0.36513026 0.3749499  0.38476954 0.39458918 0.41422846
 0.4240481  0.43386774 0.45350701 0.44368737 4.62685371 4.63667335
 4.6759519  4.64649299 4.65631263 4.66613226 4.58757515 4.59739479
 4.60721443 4.61703407 0.50260521 0.49278557 0.46332665 0.4829

Notemos que en todas las pruebas los vectores convergieron. El número de iteraciones para llegar al mínimo depende del punto inicial. En el primer ejercicio tuvimos que en el primer caso convergió en 69 iteraciones y con otro punto inicial fue con una sola iteración.


## Ejercicio 2 (5 puntos)

Programar el método de descenso máximo con tamaño de paso seleccionado
por la estrategia de backtracking:


**Algoritmo de descenso máximo con backtracking:**

Dada una función $f: \mathbb{R}^n \rightarrow \mathbb{R}$, su gradiente
$g: \mathbb{R}^n \rightarrow \mathbb{R}^n$, 
un punto inicial $x_0$, un número  máximo de iteraciones $N$, 
una tolerancia $\tau>0$. Fijar $k=0$ y repetir los siguientes pasos:

1. Calcular el gradiente en el punto $x_k$:

   $$ g_k = \nabla f(x_k) = g(x_k) $$
2. Si $\|g_k\|<\tau$, $x_k$ es un aproximadamente un punto estacionario, 
   por lo que hay que hacer $res=1$ y terminar el ciclo.
3. Eligir la dirección de descenso como $p_k = - g_k$.
4. Calcular el tamaño de paso $\alpha_k$ mediante la estrategia de backtraking,
   usando el algoritmo que describe más adelante.
5. Calcular el siguiente punto de la secuencia como
   $$x_{k+1} = x_k + \alpha_k p_k $$
6. Si ${k+1}>N$, hacer $res=0$ y terminar. 
7. Si no, hacer $k = k+1$ y volver el paso 1.
8. Devolver el punto $x_k$, $f_k= f(x_k)$, $g_k$, $k$ y $res$.

---

** Algoritmo de backtracking **

Backtracking($f$, $f_k$, $g_k$, $x_k$, $p_k$, $\alpha_{ini}$, $\rho$, $c$)

El algoritmo recibe la función $f$, el punto $x_k$, $f_k = f(x_k)$, la dirección de descenso $p_k$,
un valor inicial $\alpha_{ini}$, $\rho \in (0,1)$, $c \in (0,1)$.

Fijar  $\alpha = \alpha_{ini}$  y repetir los siguientes pasos:

1.  Si se cumple la condición
    $$ f(x_k+\alpha p_k) \leq f_k + c \alpha g_k^\top p_k, $$ 
    terminar el ciclo devolviendo $\alpha$.
2.  Hacer $\alpha = \rho \alpha$ y regresar al paso anterior.

---

1. Escriba una función que implementa el algoritmo de backtracking. 
2. Escriba la función que implementa el algoritmo de máximo
   descenso con búsqueda inexacta, usando backtraking. 
   Tiene que recibir como paramétros todos los elementos que se 
   listaron para ambos algoritmos.
3. Escriba una función para probar el funcionamiento del método
   de descenso máximo. Esta función debe recibir la función
   $f$, la función $g$ que devuelve su gradiente, el punto inicial $x_0$, el número  
   máximo de iteraciones $N$, la tolerancia $\tau>0$ y
   el factor $\rho$ del algoritmo de backtracking.
   
* Fijar los parámetros $\alpha_{ini}=2$ y $c=0.0001$ del algoritmo de backtracking.
* Ejecutar la función del Inciso 2.
* Dependiendo del valor de la variable $res$, imprima un mensaje que diga que el 
  algoritmo convergió ($res=1$) o no ($res=0$)
* Imprimir $k$, $x_k$, $f_k$ y la norma de $g_k$.

4. Pruebe la función del Inciso 3  usando $N=10000$, $\rho=0.8$, la tolerancia
   $\tau = \epsilon_m^{1/3}$, donde $\epsilon_m$ es el
   épsilon de la máquina. Aplique esta función  a:
* La función de Rosenbrock, descrita en la Tarea 3, usando como punto inicial $x_0= (-1.2, 1)$ y $x_0= (-12, 10)$.
  Como referencia, el minimizador de la función  es $x_* = (1,1)$.
  
5. Repita el inciso anterior con $\rho=0.5$.   

   
### Solución:

In [43]:
# En esta celda puede poner el código de las funciones
# o poner la instrucción para importarlas de un archivo .py

def Backtraking(f, fk, gk, xk, pk, a0, rho, c):
    a = a0
    while f(xk + a*pk) > fk +c*a*gk.T@pk:
        a = rho*a
    return a


def DescMAxBack(f, g, x0, N, tol, a0, rho, c):
    xk = x0      
    for k in range(N):
      gk = g(xk)
      norm = LA.norm(gk)
      if norm < tol:
          res = 1
          break
      else:
          pk = -gk
          fk = f(xk)
          ak = Backtraking(f, fk, gk, xk, pk, a0, rho, c)
          xk = xk + ak*pk
          if k+1 >= N:
              res = 0
              break
    return xk, fk, gk, k, res

    

In [44]:
# Lectura de datos y pruebas realizadas

def Probar2(f, g, x0, N, tol, rho):
    a0 = 2
    c = 0.0001
    print('N:', N, 'x0:', x0, 'rho:', rho)
    xk, fk, gk, k, res = DescMAxBack (f, g, x0, N, tol, a0, rho, c)
    if res == 1:
        print('¡Convergió! :)')
    else:
        print('¡No convergió! :(')
    print('Valor k:', k)
    print('xk:', xk)
    print('fk:', fk)
    print('||gk||:', LA.norm(gk))
    
def function(x):
    xT = x.T
    return 100 * (xT[1] - xT[0]**2)**2 + (1-xT[0])**2
    
def gradient(x):
    xT = x.T
    return np.array([-400*xT[0]*(xT[1]- xT[0]**2) -2*(1-xT[0]), 200*(xT[1]-xT[0]**2)])

In [45]:
from numpy.core.fromnumeric import partition
#Pruebas
eps = sys.float_info.epsilon
tol = eps**(1/3)

x0s = [np.array([-1.2,1]) , np.array([-12,10])]

rhos = [0.8, 0.5]

Ns = [10000, 50000]

print("Descenso máximo con backtracking. \n")

for x0 in x0s:
    for rho in rhos:
        for N in Ns:
            Probar2(function, gradient, x0, N, tol, rho)
            print('\n')


Descenso máximo con backtracking. 

N: 10000 x0: [-1.2  1. ] rho: 0.8
¡No convergió! :(
Valor k: 9999
xk: [1.00079208 1.00158391]
fk: 6.284994359483611e-07
||gk||: 0.0019653292817284995


N: 50000 x0: [-1.2  1. ] rho: 0.8
¡Convergió! :)
Valor k: 17005
xk: [1.00000294 1.0000059 ]
fk: 8.709467898400525e-12
||gk||: 5.999000737744113e-06


N: 10000 x0: [-1.2  1. ] rho: 0.5
¡No convergió! :(
Valor k: 9999
xk: [0.99998357 0.99996704]
fk: 2.7141545777021076e-10
||gk||: 2.377226272021971e-05


N: 50000 x0: [-1.2  1. ] rho: 0.5
¡Convergió! :)
Valor k: 11525
xk: [0.99999522 0.99999044]
fk: 2.286837854150557e-11
||gk||: 6.004300577326651e-06


N: 10000 x0: [-12  10] rho: 0.8


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in long_scalars


¡No convergió! :(
Valor k: 9999
xk: [ 3.92075554 15.37404809]
fk: 8.531123402708898
||gk||: 2.823173926047487


N: 50000 x0: [-12  10] rho: 0.8
¡No convergió! :(
Valor k: 49999
xk: [3.13859163 9.85233986]
fk: 4.573871935931183
||gk||: 2.0410061941286526


N: 10000 x0: [-12  10] rho: 0.5
¡No convergió! :(
Valor k: 9999
xk: [2.85989511 8.18266786]
fk: 3.460691855385309
||gk||: 1.121519863216604


N: 50000 x0: [-12  10] rho: 0.5
¡Convergió! :)
Valor k: 42287
xk: [1.00000469 1.00000939]
fk: 2.206627769944216e-11
||gk||: 6.021268237021556e-06




Notemos que para $N=10000$, el método no converge para ningún punto inicial.

Para un $N$ más grande sí converge. Vemos que este método es más inexacto que el de la tarea 3. Es muy útil saber el punto donde converge para que converja rapidamente y elegir esto para nuestra conveniencia. Sin embargo, en la práctica este método puede ser más costoso computacionalmente.